# Lyrics.com artist classifier




**Goals:**  
In this project, we will build a text classification model on song lyrics. 
The task is to predict the artist from a piece of text. 
To train such a model, you first need to collect your own lyrics dataset:

- Download a HTML page with links to songs

- Extract hyperlinks of song pages

- Download and extract the song lyrics

- Vectorize the text using the Bag Of Words method

- train a classification model that predicts the artist from a piece of text

- refactor the code into functions

- Write a simple command-line interface for the program

- upload your code to GitHub



## Lyrics download

- dev functions that:
    1. takes artist name, checks if exists, 
    2. grabs all lyrics links
    3. downloads lyrics

### Artist Checking

In [1]:
import lyrics_fun as lf

In [2]:
arep = lf.get_artist("Mô", True)
arep

Artist: Mô ----- Status: 200
    Several options for artist; picking first


{'base_url': 'https://www.lyrics.com',
 'artist': 'Mô',
 'url': 'https://www.lyrics.com/artist/M%C3%B4/',
 'url_refined': 'https://www.lyrics.com/artist/Mo/106563',
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': True}

In [3]:
arep = lf.get_artist("twiddieyiszl", True)
arep

Artist: twiddieyiszl ----- Status: 200
    No unique artist found


{'base_url': 'https://www.lyrics.com',
 'artist': 'twiddieyiszl',
 'url': 'https://www.lyrics.com/artist/twiddieyiszl/',
 'url_refined': None,
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': False}

In [4]:
# test with single hit
req = lf.get_artist("Olivia Rodrigo", True)
arep = req['response']
aurl =req['url']
lf.refine_artist_link(arep, aurl, 'lyrics.com/', False)


Artist: Olivia Rodrigo ----- Status: 200
    Found unique artist


'https://www.lyrics.com/artist/Olivia%20Rodrigo/'

In [5]:
# test with single hit
req = lf.get_artist("Kiss", False)
arep = req['response']
aurl =req['url']
lf.refine_artist_link(arep, aurl, 'lyrics.com/', True)


    Found unique artist


'https://www.lyrics.com/artist/Kiss/'

In [6]:
# test with multi hit
req = lf.get_artist("sun", True)
arep = req['response']
aurl =req['url']
lf.refine_artist_link(arep, aurl, 'lyrics.com/', True)


Artist: sun ----- Status: 200
    Several options for artist; picking first
    Several options for artist; picking first


'lyrics.com//artist/Sun/5559'

In [7]:
# test with no hit
req = lf.get_artist("gibidigbisij2", True)
arep = req['response']
aurl =req['url']
lf.refine_artist_link(arep, aurl, 'lyrics.com/', True)


Artist: gibidigbisij2 ----- Status: 200
    No unique artist found
    No unique artist found


In [8]:
artist = lf.get_artist("Olivia Rodrigo", True)
artist

Artist: Olivia Rodrigo ----- Status: 200
    Found unique artist


{'base_url': 'https://www.lyrics.com',
 'artist': 'Olivia Rodrigo',
 'url': 'https://www.lyrics.com/artist/Olivia%20Rodrigo/',
 'url_refined': 'https://www.lyrics.com/artist/Olivia%20Rodrigo/',
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': True}

### Grab lyric links


- check if `exists_on_site = True`
- grab all song links

In [1]:
import lyrics_fun as lf

artist = lf.get_artist("Sun", True)
artist_links = lf.extract_lyric_links(artist, drop_duplicates = True, drop_instrumentals = True, drop_similar=True, verbose=True)
len(artist_links['links'])

Artist: Sun ----- Status: 200
    Several options for artist; picking first
    Dropped 20 duplicated lyric links
    Dropped 1 instrumental lyric link(s)
    Dropped 0 Remix/Acoustic lyric link(s)


6

In [10]:
artist

{'base_url': 'https://www.lyrics.com',
 'artist': 'Sun',
 'url': 'https://www.lyrics.com/artist/Sun/',
 'url_refined': 'https://www.lyrics.com/artist/Sun/5559',
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': True}

In [11]:
artist_links

{'base_url': 'https://www.lyrics.com',
 'artist': 'Sun',
 'url': 'https://www.lyrics.com/artist/Sun/',
 'url_refined': 'https://www.lyrics.com/artist/Sun/5559',
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': True,
 'links': ['https://www.lyrics.com/lyric/7067990/Sun/I+Had+a+Choice',
  'https://www.lyrics.com/lyric/3611604/Sun/Sun+Is+Here',
  'https://www.lyrics.com/lyric/4595999/Sun/Force+of+Nature',
  'https://www.lyrics.com/lyric/4596655/Sun/My+Woman',
  'https://www.lyrics.com/lyric-lf/6468010/Sun/Sailor',
  'https://www.lyrics.com/lyric-lf/1361310/Sun/Ms.+Communication']}

### Download lyrics

Options for dropping duplicates and instrumentals

In [12]:
extract_test = lf.extract_lyric(artist_links=artist_links, verbose = False)

In [14]:
lf.pd.DataFrame({'title' : extract_test['lyric_title'], 'lyric': extract_test['lyric_text']})

,title,lyric
0,I Had a Choice,"Holding hands, strolling through the park Swee..."
1,Sun Is Here,You come in numbers to feel a groove I have a ...
2,Force of Nature,It's gettin kinda heavy! Yeah yeah It's gettin...
3,My Woman,Little boy was blue because he knew that soon ...
4,Sailor,Sailor What do you want from me? Sailor I'm no...
5,Ms. Communication,I just wanna put it all on the table Maybe we ...


In [1]:
import lyrics_fun as lf


fail_artist = lf.get_artist("gibidigbisij2", True)
fail_links = lf.extract_lyric_links(fail_artist, drop_duplicates = True, drop_instrumentals = True, drop_similar=True, verbose=True)


lf.extract_lyric(artist_links=fail_links, verbose = False)




Artist: gibidigbisij2 ----- Status: 200
    No unique artist found
    Can only proceed with an artist listed at lyrics.com, ensure exists_on_site is True. Returning None
    Can only proceed with an artist listed at lyrics.com, ensure lyric link extraction was successful. Returning None


## Improvements

- Scan for non-english text?! (found korean symbols in one song from Sun)
- Strip symbols/punctuation/notation
- strip typical lyrical notation/syntax, like "Verse 2", "Hook"
- **implement max_lyrics** in extract_lyrics() with a reasonable default value of e.g., 50
- ~~implement `drop_similar` and use regex to search for patterns: "acoustic|remix"~~ (needs more testing)

## Next Steps:

- ~~dev wrapper function to run all lyric steps~~
- dev wrapper to fit logReg model
    - make X_y df (wrapper for pd.DataFrame)
- dev wrapper with CLI

In [1]:
import lyrics_fun as lf

lyrics = lf.process_artist("Reel Big Fish")

Artist: Reel Big Fish ----- Status: 200
    Found unique artist
    Dropped 325 duplicated lyric links
    Dropped 2 instrumental lyric link(s)
    Dropped 20 Remix/Acoustic lyric link(s)


In [7]:
lyrics.keys()

dict_keys(['base_url', 'artist', 'url', 'url_refined', 'response', 'status_code', 'exists_on_site', 'lyric_link', 'lyric_title', 'lyric_text'])

In [2]:
import lyrics_fun as lf
lyrics = lf.process_artist(artist = "OneRepublic", verbose = True)
lf.make_lyric_df(lyrics)

Artist: OneRepublic ----- Status: 200
    Found unique artist
    Dropped 537 duplicated lyric links
    Dropped 0 instrumental lyric link(s)
    Dropped 14 Remix/Acoustic lyric link(s)


,artist,title
0,OneRepublic,Woah Woah You bring an energy I've never felt ...
1,OneRepublic,I'm holding on your rope Got me ten feet off t...
2,OneRepublic,Never could imagine the way the story happened...
3,OneRepublic,"She said, every time I close my eyes I feel li..."
4,OneRepublic,"This town is colder now, I think it's sick of ..."
...,...,...
131,OneRepublic,I forgot about you I forgot about what you sai...
132,OneRepublic,Tell me about your day Tell me everything that...
133,OneRepublic,"Some days you're alone, yeah Some days this do..."
134,OneRepublic,"Some days, I'm treadin' the water And feel lik..."


In [4]:
lyrics['url_refined']

'https://www.lyrics.com/artist/OneRepublic/'

In [6]:
lyric_df = lf.make_lyric_df(lyrics)
lyric_df.shape

(136, 2)